In [1]:
import numpy as np
from tifffile import imread, imwrite 
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import glob
sys.path.append("../NEAT")
from  NEATUtils import Augmentation
from NEATUtils import MovieCreator, npzfileGenerator
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile
import scipy.ndimage as ndi
import csv
from stardist.models import StarDist2D
from csbdeep.models import Config, CARE
from csbdeep.utils import normalize
from skimage.filters import threshold_otsu
import cv2
from skimage.measure import regionprops
from skimage import measure, morphology
from skimage.filters import sobel
from skimage.segmentation import watershed
import matplotlib.pyplot  as plt

Using TensorFlow backend.


In [2]:
#Normal : Label 0 Apoptosis : Label 1 Division : Label 2 Macrocheate : Label 3 Non MatureP1 : Label 4 Mature Event: Label 5

In [3]:
ModelDirectory = '/home/sancere/VarunNewton/CurieDeepLearningModels/O-NEATweights/'

DenoisingModel = 'BorialisS1S2FlorisMidNoiseModel'
SegmentationMaskModel = 'DrosophilaMaskSegmentationCARE'
SegmentationStarModel = 'DrosophilaSegmentationSmartSeeds'

model = StarDist2D(config = None, name = SegmentationStarModel, basedir = ModelDirectory)
UnetModel = CARE(config = None, name = SegmentationMaskModel, basedir = ModelDirectory)
NoiseModel = CARE(config = None, name = DenoisingModel, basedir = ModelDirectory)






Loading network weights from 'weights_best.h5'.






Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


In [20]:

Dir = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neat/StaticNonMatureP1EventTrainingDataBin2/ShiftUY/'
cellX = 0.5
cellY = 0.314814814814815
trainlabel = 4

In [21]:
axis_norm = (0,1)
TotalCategories = 6
Label = np.zeros([TotalCategories + 4])
Label[trainlabel] = 1

Subdir = next(os.walk(Dir))
for x in Subdir[1]:
    currentdir = Dir  + x
    Path(currentdir + '/Segmentation/').mkdir(exist_ok = True)
    Raw_path = os.path.join(currentdir, '*tif')
    X = glob.glob(Raw_path)
    for fname in X:
        image = imread(fname)
        #Get Name
        Name = os.path.basename(os.path.splitext(fname)[0])



        x = image

        originalX = x.shape[0]
        originalY = x.shape[1]
       
        #Stardist Prediction

        scale_percent = 400 # percent of original size
        width = int(x.shape[1] * scale_percent / 100)
        height = int(x.shape[0] * scale_percent / 100)
        dim = (width, height)
        # resize image
        x = cv2.resize(x, dim)
        x = normalize(x,1,99.8,axis=axis_norm)
       

        MidImage, details = model.predict_instances(x)
       
        

        segimage = MidImage
       

        #Save different method segmentation

        imwrite((currentdir + '/Segmentation/'  + Name+ '.tif' ) , segimage.astype('float32'))
     
        smallimg = segimage.astype('uint16')
        centralLabel = smallimg[int(smallimg.shape[0]*cellY), int(smallimg.shape[1]*cellX)]
        
        
        if len(regionprops(smallimg)) > 0:
         for region in regionprops(smallimg):
           
            if region.label == centralLabel and centralLabel > 0:
              minr, minc, maxr, maxc = region.bbox
              
              center = region.centroid
              height = 0.25 * abs(maxc - minc)
              width = 0.25 * abs(maxr - minr)
            if centralLabel == 0:
            
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
               
        else:
               center = (smallimg.shape[0]*cellY,smallimg.shape[1]*cellX)
               height = 10
               width = 10 
            
              
        print(centralLabel, Name, 0.25 * center[0]/originalY, 0.25 * center[1]/originalX, height/originalY, width/originalX)
            
        Label[TotalCategories] = 0.25 * center[1]/originalX
        Label[TotalCategories + 1] = 0.25 * center[0]/originalY
       
        Label[TotalCategories + 2] = height/originalY
        Label[TotalCategories + 3] = width/originalX
        
        writer = csv.writer(open(currentdir + '/' + (Name) + 'DetectionLabel'  +".csv", "w"))
   
        for l in Label : writer.writerow ([l])
        
        

0 39Bin2 0.314814814814815 0.5 0.18518518518518517 0.18518518518518517
13 51Bin2 0.32115414169356077 0.5153296449976947 0.09259259259259259 0.09259259259259259
19 30Bin2 0.35582322551662177 0.44360836577817714 0.17592592592592593 0.25
26 36Bin2 0.34574915824915825 0.4898989898989899 0.13425925925925927 0.125
22 19Bin2 0.25140278439468533 0.5937262894147125 0.21296296296296297 0.27314814814814814
23 54Bin2 0.2950846080573766 0.4415307894884294 0.14351851851851852 0.1574074074074074
8 3Bin2 0.312537037037037 0.5574444444444444 0.12962962962962962 0.05555555555555555
0 41Bin2 0.314814814814815 0.5 0.18518518518518517 0.18518518518518517
9 14Bin2 0.3178750784682988 0.4906230382925298 0.0787037037037037 0.09259259259259259
8 15Bin2 0.32407407407407407 0.48524737423991154 0.09259259259259259 0.08333333333333333
17 2Bin2 0.2897549937578028 0.4692571785268414 0.10185185185185185 0.12037037037037036
24 31Bin2 0.3340918197950656 0.49204304768446966 0.1388888888888889 0.12962962962962962
22 11Bin

19 12Bin2 0.3165849673202614 0.4938980800653595 0.1527777777777778 0.19444444444444445
24 25Bin2 0.3277669391799826 0.48406726124117433 0.13425925925925927 0.13425925925925927
12 22Bin2 0.312359523111502 0.5082575979673605 0.14351851851851852 0.07407407407407407
15 48Bin2 0.2889859556526223 0.6237500126389015 0.3333333333333333 0.3055555555555556
0 24Bin2 0.314814814814815 0.5 0.18518518518518517 0.18518518518518517
10 28Bin2 0.44807239057239057 0.5596156004489338 0.18055555555555555 0.2824074074074074
22 18Bin2 0.3252767134951043 0.5088229033631332 0.11574074074074074 0.09722222222222222
25 7Bin2 0.2719867633956981 0.4866122565864834 0.14351851851851852 0.10648148148148148
21 21Bin2 0.42705229520231464 0.5848033711908566 0.2777777777777778 0.25462962962962965
11 29Bin2 0.3754711092266742 0.3721929741910662 0.4027777777777778 0.32407407407407407
22 34Bin2 0.32503327852165065 0.458078850520711 0.1712962962962963 0.09722222222222222
18 38Bin2 0.38415817564289184 0.6268788074828832 0.3379